In [36]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [37]:
data = pd.read_csv('data/train.csv')
kaggle_test = pd.read_csv('data/test.csv')

In [38]:
train, test = train_test_split(data, test_size=0.3, random_state=42)

In [39]:
train.isnull().sum()

PassengerId       0
HomePlanet      140
CryoSleep       154
Cabin           142
Destination     122
Age             129
VIP             153
RoomService     114
FoodCourt       122
ShoppingMall    146
Spa             122
VRDeck          129
Name            138
Transported       0
dtype: int64

## Fill gaps

In [40]:
numeric_fields = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categ_fields = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

Fill numeric features with median values

In [41]:
num_imputer = SimpleImputer(strategy='median')
train[numeric_fields] = num_imputer.fit_transform(train[numeric_fields])
test[numeric_fields] = num_imputer.transform(test[numeric_fields])
kaggle_test[numeric_fields] = num_imputer.transform(kaggle_test[numeric_fields])

Fill categorial features with most frequent values

In [42]:
cat_imputer = SimpleImputer(strategy='most_frequent')
train[categ_fields] = cat_imputer.fit_transform(train[categ_fields])
test[categ_fields] = cat_imputer.transform(test[categ_fields])
kaggle_test[categ_fields] = cat_imputer.transform(kaggle_test[categ_fields])

In [43]:
train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin           142
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            138
Transported       0
dtype: int64

Fill all null names with default name John Doe

In [44]:
train["Name"] = train["Name"].fillna("John Doe")
test["Name"] = test["Name"].fillna("John Doe")
kaggle_test["Name"] = kaggle_test["Name"].fillna("John Doe")

Fill all null cabin values as: Deck X Room 0 Side X

In [45]:
train["Cabin"] = train["Cabin"].fillna("X/0/X")
test["Cabin"] = test["Cabin"].fillna("X/0/X")
kaggle_test["Cabin"] = kaggle_test["Cabin"].fillna("X/0/X")

## Normalization

But firstly, we need to split cabin into 3 columns as it's more useful. Also otherwise cabin number can't be normalized easily.

In [46]:
train[["Deck", "Room", "Side"]] = train["Cabin"].str.split("/", expand=True)
train = train.drop("Cabin", axis=1)
test[["Deck", "Room", "Side"]] = test["Cabin"].str.split("/", expand=True)
test = test.drop("Cabin", axis=1)
kaggle_test[["Deck", "Room", "Side"]] = kaggle_test["Cabin"].str.split("/", expand=True)
kaggle_test = kaggle_test.drop("Cabin", axis=1)

In [47]:
numeric_fields = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Room']

Now normalization via some scaler

fit normalizer

In [48]:
scaler = MinMaxScaler()
scaler.fit(train[numeric_fields])

MinMaxScaler()

now transofrm data based on fitting

In [49]:
train[numeric_fields] = scaler.transform(train[numeric_fields])
test[numeric_fields] = scaler.transform(test[numeric_fields])
kaggle_test[numeric_fields] = scaler.transform(kaggle_test[numeric_fields])

## Replace categories with OHE

In [50]:
categ_fields = ['HomePlanet', 'CryoSleep', 'Destination', 'Deck', 'Side']

In [51]:
train_encoded = pd.get_dummies(train, columns=categ_fields)
test_encoded = pd.get_dummies(test, columns=categ_fields)
kaggle_test_encoded = pd.get_dummies(kaggle_test, columns=categ_fields)

I checked and found out that no Deck_T present in the test, so form uniform data we need to make sure columns match

In [52]:
for c in train_encoded.columns:
    if (c not in test_encoded.columns):
        test_encoded[c] = 0

for c in test_encoded.columns:
    if (c not in train_encoded.columns):
        train_encoded[c] = 0

for c in test_encoded.columns:
    if (c not in kaggle_test_encoded.columns):
        kaggle_test_encoded[c] = 0

In [53]:
test_encoded.head()

,PassengerId,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_X,Side_P,Side_S,Side_X,Deck_T
304,0337_02,0.240506,False,0.029106,0.012589,0.026988,0.000162,0.043799,Weros Perle,True,...,False,False,False,True,False,False,False,True,False,0
2697,2891_01,0.227848,False,0.000279,0.032608,0.000000,0.000000,0.000041,Gleney Ortinericey,False,...,False,False,False,False,True,False,False,True,False,0
8424,8998_01,0.518987,False,0.000000,0.000000,0.000000,0.000000,0.000000,Gerry Englence,False,...,False,False,False,False,True,False,False,True,False,0
1672,1771_01,0.443038,False,0.000000,0.012192,0.018560,0.000000,0.000000,Antone Cardner,True,...,False,False,False,False,True,False,True,False,False,0
8458,9034_02,0.544304,False,0.000000,0.000000,0.000000,0.000000,0.000000,Errairk Crakete,True,...,False,True,False,False,False,False,True,False,False,0


In [ ]:
train_encoded.to_csv('data/train_preprocessed.csv', index=False)
test_encoded.to_csv('data/test_preprocessed.csv', index=False)
kaggle_test_encoded.to_csv('data/kaggle_test_preprocessed.csv', index=False)